## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

## Mirroring

-  Checks if Primary/Secondary or Principal/Mirror (works on all versions) (Query 1)
-  Checks if Mirroring configured (Query 2)
-  Mirroring information, Database, state, partner info (Query 3)

In [ ]:
/* Primary/Secondary or Mirroring Principle or Mirror.sql  #SQL */

GO
DECLARE @SQLServerInstanceVersion INT
SELECT  @SQLServerInstanceVersion=FLOOR(SUBSTRING(CAST(SERVERPROPERTY('ProductVersion')AS VARCHAR(10)),1,4)) 


IF (@SQLServerInstanceVersion < 11)
EXEC('
SELECT @@ServerName AS ServerName
	,a.NAME
	,COALESCE(b.mirroring_role_desc, '''') AS mirroring_role_desc
FROM sys.databases a
LEFT JOIN sys.database_mirroring b ON a.database_id = b.database_id
WHERE Name NOT IN(''master'',''1_Admin'',''1_Admin_Mirror_Data'',''model'',''msdb'',''tempdb'',''DBATools'')
')

ELSE
EXEC('
SELECT 
	@@SERVERNAME AS serverName
	,d.name 
,(CASE 
		
		WHEN is_primary_replica IS NULL
		THEN ''''
		WHEN is_primary_replica = 1
			THEN ''PRIMARY''
		WHEN is_primary_replica = 0
		THEN''SECONDARY''
		END) AS mirroring_role_desc
FROM Master.sys.databases d
LEFT JOIN master.sys.dm_hadr_database_replica_states r ON d.database_id = r.database_id AND database_state_desc = ''ONLINE''
WHERE a.Name NOT IN(''master'',''1_Admin'',''1_Admin_Mirror_Data'',''model'',''msdb'',''tempdb'',''DBATools'')
')



- #2 Checks if Mirroring configured  (Query 2)

In [ ]:
/*Checks if Mirroring configured  #SQL */
SELECT A.NAME
	,CASE 
		WHEN B.mirroring_state IS NULL
			THEN 'Mirroring not configured'
		ELSE 'Mirroring configured'
		END AS MirroringState
FROM sys.databases A
INNER JOIN sys.database_mirroring B ON A.database_id = B.database_id
WHERE a.database_id > 4
ORDER BY A.NAME

## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

- #2 Mirroring information, Database, state, partner info  (Query 3)

In [ ]:
/* Mirroring information, Database, state, partner*/
SELECT @@SERVERNAME AS ServerName
	,database_id
	,d.NAME
	,mirroring_partner_name
	,mirroring_witness_name
	,mirroring_guid
	,mirroring_state
	,mirroring_state_desc
	,mirroring_role
	,mirroring_role_desc
	,mirroring_role_sequence
	,mirroring_safety_level
	,mirroring_safety_level_desc
	,mirroring_safety_sequence
	,mirroring_partner_instance
	,mirroring_witness_state
	,mirroring_witness_state_desc
	,mirroring_failover_lsn
	,mirroring_connection_timeout
	,mirroring_redo_queue
	,mirroring_redo_queue_type
	,mirroring_end_of_log_lsn
	,mirroring_replication_lsn
FROM sys.database_mirroring m
JOIN master.dbo.sysdatabases d ON m.database_id = d.dbid